In [ ]:
!pip install matminer
#modul yang digunakan untuk mining data meterial

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval

# Mining Anorganic Material
Mendapatkan material anorganik dengan ID yang berbeda yang mengandung besaran energy bandgap. Data tersebut diambil dari modul Matminer

In [ ]:
from matminer.datasets.convenience_loaders import load_dielectric_constant
df = load_dielectric_constant()
df.head()

,material_id,formula,nsites,space_group,volume,structure,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric
0,mp-441,Rb2Te,3,225,159.501208,"[[1.75725875 1.2425695 3.04366125] Rb, [5.271...",1.88,"[[3.44115795, -3.097e-05, -6.276e-05], [-2.837...","[[6.23414745, -0.00035252, -9.796e-05], [-0.00...",1.86,3.44,6.23,False
1,mp-22881,CdCl2,3,166,84.298097,"[[0. 0. 0.] Cd, [ 4.27210959 2.64061969 13.13...",3.52,"[[3.34688382, -0.04498543, -0.22379197], [-0.0...","[[7.97018673, -0.29423886, -1.463590159999999]...",1.78,3.16,6.73,False
2,mp-28013,MnI2,3,164,108.335875,"[[0. 0. 0.] Mn, [-2.07904300e-06 2.40067320e+...",1.17,"[[5.5430849, -5.28e-06, -2.5030000000000003e-0...","[[13.80606079, 0.0006911900000000001, 9.655e-0...",2.23,4.97,10.64,False
3,mp-567290,LaN,4,186,88.162562,[[-1.73309900e-06 2.38611186e+00 5.95256328e...,1.12,"[[7.09316738, 7.99e-06, -0.0003864700000000000...","[[16.79535386, 8.199999999999997e-07, -0.00948...",2.65,7.04,17.99,False
4,mp-560902,MnF2,6,136,82.826401,"[[1.677294 2.484476 2.484476] Mn, [0. 0. 0.] M...",2.87,"[[2.4239622, 7.452000000000001e-05, 6.06100000...","[[6.44055613, 0.0020446600000000002, 0.0013203...",1.53,2.35,7.12,False


# Add Material Properties (Featurizes) on Dataset

menambahkan sifat-sifat material yang didapatkan dari matminer berdasarkan ID material pada data sebelumnya, sehingga didapatkan banyak fitur yang pada tahap selanjutnya akan diseleksi

In [ ]:
from matminer.datasets import load_dataset
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.composition import ElementProperty, Stoichiometry, ValenceOrbital, IonProperty
from matminer.featurizers.structure import (SiteStatsFingerprint, StructuralHeterogeneity,
                                            ChemicalOrdering, StructureComposition, MaximumPackingEfficiency)
from matminer.featurizers.conversions import DictToObject


#nama-nama jenis fitur/ kolom yang ditambahkan berdasarkan ID materialnya 
featurizer = MultipleFeaturizer([
    SiteStatsFingerprint.from_preset("CoordinationNumber_ward-prb-2017"),
    StructuralHeterogeneity(),
    ChemicalOrdering(),
    MaximumPackingEfficiency(),
    SiteStatsFingerprint.from_preset("LocalPropertyDifference_ward-prb-2017"),
    StructureComposition(Stoichiometry()),
    StructureComposition(ElementProperty.from_preset("magpie")),
    StructureComposition(ValenceOrbital(props=['frac'])),
    StructureComposition(IonProperty(fast=True))])

In [ ]:
data = df

dto = DictToObject(target_col_id='structure', overwrite_data=True)
data = dto.featurize_dataframe(data, 'structure')
data.head

DictToObject: 100%|██████████| 1056/1056 [00:00<00:00, 3468.02it/s]


<bound method NDFrame.head of      material_id  ...                                          structure
0         mp-441  ...  [[1.75725875 1.2425695  3.04366125] Rb, [5.271...
1       mp-22881  ...  [[0. 0. 0.] Cd, [ 4.27210959  2.64061969 13.13...
2       mp-28013  ...  [[0. 0. 0.] Mn, [-2.07904300e-06  2.40067320e+...
3      mp-567290  ...  [[-1.73309900e-06  2.38611186e+00  5.95256328e...
4      mp-560902  ...  [[1.677294 2.484476 2.484476] Mn, [0. 0. 0.] M...
...          ...  ...                                                ...
1051   mp-568032  ...  [[0. 0. 0.] Cd, [2.9560375 0.        3.03973  ...
1052   mp-696944  ...  [[2.068917   3.58317965 3.70992025] La, [4.400...
1053    mp-16238  ...  [[1.35965225 0.96141925 2.354987  ] Li, [2.719...
1054     mp-4405  ...  [[0.       2.808758 2.808758] Rb, [2.808758 2....
1055     mp-3486  ...  [[-1.89006800e-06  2.56736395e+00  1.32914373e...

[1056 rows x 13 columns]>

In [ ]:
data.head()

,material_id,formula,nsites,space_group,volume,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric,structure
0,mp-441,Rb2Te,3,225,159.501208,1.88,"[[3.44115795, -3.097e-05, -6.276e-05], [-2.837...","[[6.23414745, -0.00035252, -9.796e-05], [-0.00...",1.86,3.44,6.23,False,"[[1.75725875 1.2425695 3.04366125] Rb, [5.271..."
1,mp-22881,CdCl2,3,166,84.298097,3.52,"[[3.34688382, -0.04498543, -0.22379197], [-0.0...","[[7.97018673, -0.29423886, -1.463590159999999]...",1.78,3.16,6.73,False,"[[0. 0. 0.] Cd, [ 4.27210959 2.64061969 13.13..."
2,mp-28013,MnI2,3,164,108.335875,1.17,"[[5.5430849, -5.28e-06, -2.5030000000000003e-0...","[[13.80606079, 0.0006911900000000001, 9.655e-0...",2.23,4.97,10.64,False,"[[0. 0. 0.] Mn, [-2.07904300e-06 2.40067320e+..."
3,mp-567290,LaN,4,186,88.162562,1.12,"[[7.09316738, 7.99e-06, -0.0003864700000000000...","[[16.79535386, 8.199999999999997e-07, -0.00948...",2.65,7.04,17.99,False,[[-1.73309900e-06 2.38611186e+00 5.95256328e...
4,mp-560902,MnF2,6,136,82.826401,2.87,"[[2.4239622, 7.452000000000001e-05, 6.06100000...","[[6.44055613, 0.0020446600000000002, 0.0013203...",1.53,2.35,7.12,False,"[[1.677294 2.484476 2.484476] Mn, [0. 0. 0.] M..."


In [ ]:
%%time
print('Total number of features:', len(featurizer.featurize(data['structure'][0])))
print('Number of sites in structure:', len(data['structure'][0]))

Total number of features: 273
Number of sites in structure: 3
CPU times: user 314 ms, sys: 1.6 ms, total: 315 ms
Wall time: 321 ms


In [ ]:
%%time
X = featurizer.featurize_dataframe(data, col_id='structure',ignore_errors=True)

MultipleFeaturizer: 100%|██████████| 1056/1056 [24:18<00:00,  1.38s/it]


CPU times: user 1.63 s, sys: 482 ms, total: 2.12 s
Wall time: 27min 54s


In [ ]:
X.head()

,material_id,formula,nsites,space_group,volume,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric,structure,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,mean CN_VoronoiNN,avg_dev CN_VoronoiNN,mean absolute deviation in relative bond length,max relative bond length,min relative bond length,minimum neighbor distance variation,maximum neighbor distance variation,range neighbor distance variation,mean neighbor distance variation,avg_dev neighbor distance variation,mean absolute deviation in relative cell size,mean ordering parameter shell 1,mean ordering parameter shell 2,mean ordering parameter shell 3,max packing efficiency,minimum local difference in Number,maximum local difference in Number,range local difference in Number,mean local difference in Number,avg_dev local difference in Number,minimum local difference in MendeleevNumber,maximum local difference in MendeleevNumber,range local difference in MendeleevNumber,mean local difference in MendeleevNumber,...,MagpieData mean NfUnfilled,MagpieData avg_dev NfUnfilled,MagpieData mode NfUnfilled,MagpieData minimum NUnfilled,MagpieData maximum NUnfilled,MagpieData range NUnfilled,MagpieData mean NUnfilled,MagpieData avg_dev NUnfilled,MagpieData mode NUnfilled,MagpieData minimum GSvolume_pa,MagpieData maximum GSvolume_pa,MagpieData range GSvolume_pa,MagpieData mean GSvolume_pa,MagpieData avg_dev GSvolume_pa,MagpieData mode GSvolume_pa,MagpieData minimum GSbandgap,MagpieData maximum GSbandgap,MagpieData range GSbandgap,MagpieData mean GSbandgap,MagpieData avg_dev GSbandgap,MagpieData mode GSbandgap,MagpieData minimum GSmagmom,MagpieData maximum GSmagmom,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,compound possible,max ionic char,avg ionic char
0,mp-441,Rb2Te,3,225,159.501208,1.88,"[[3.44115795, -3.097e-05, -6.276e-05], [-2.837...","[[6.23414745, -0.00035252, -9.796e-05], [-0.00...",1.86,3.44,6.23,False,"[[1.75725875 1.2425695 3.04366125] Rb, [5.271...",8.000000,9.971809,1.971809e+00,9.314539,8.763594e-01,3.725711e-02,1.027943,0.944114,4.574309e-08,0.069488,6.948827e-02,0.046326,3.088368e-02,1.041667e-01,0.389073,0.079838,0.053132,0.510131,6.390728,15.000000,8.609272e+00,9.260485,3.826343e+00,36.640171,86.000,4.935983e+01,53.093448,...,0.0,0.0,0.0,1.0,2.0,1.0,1.333333,0.444444,1.0,34.763333,90.722500,55.959167,72.069444,24.870741,90.72250,0.0,0.464,0.464,0.154667,0.206222,0.000,0.0,0.00000,0.00000,0.000000,0.000000,0.0,152.0,229.0,77.0,203.333333,34.222222,229.0,0.222222,0.222222,0.555556,0.0,True,0.336084,0.074685
1,mp-22881,CdCl2,3,166,84.298097,3.52,"[[3.34688382, -0.04498543, -0.22379197], [-0.0...","[[7.97018673, -0.29423886, -1.463590159999999]...",1.78,3.16,6.73,False,"[[0. 0. 0.] Cd, [ 4.27210959 2.64061969 13.13...",6.005276,10.884861,4.879585e+00,9.258333,2.168704e+00,1.112596e-01,1.083445,0.833111,7.214204e-04,0.180224,1.795025e-01,0.120390,7.977887e-02,2.055257e-01,0.334561,0.092067,0.078179,0.359239,12.080936,31.000000,1.891906e+01,18.387291,8.408473e+00,9.352983,24.000,1.464702e+01,14.235322,...,0.0,0.0,0.0,0.0,1.0,1.0,0.666667,0.444444,1.0,19.495000,24.497500,5.002500,22.830000,2.223333,24.49750,0.0,2.493,2.493,1.662000,1.108000,2.493,0.0,0.00000,0.00000,0.000000,0.000000,0.0,64.0,194.0,130.0,107.333333,57.777778,64.0,0.230769,0.384615,0.384615,0.0,True,0.417383,0.092752
2,mp-28013,MnI2,3,164,108.335875,1.17,"[[5.5430849, -5.28e-06, -2.5030000000000003e-0...","[[13.80606079, 0.0006911900000000001, 9.655e-0...",2.23,4.97,10.64,False,"[[0. 0. 0.] Mn, [-2.07904300e-06 2.40067320e+...",6.000000,10.784332,4.784332e+00,9.189554,2.126370e+00,1.109774e-01,1.083233,0.833534,4

In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
X.to_csv('dataset_band_gap.csv')
!cp dataset_band_gap.csv "drive/My Drive/"

In [ ]:
import numpy as np
X = np.array(X)
print('Input data shape:', X.shape)

Input data shape: (1056, 273)


In [ ]:
import pandas as pd
failed = np.any(pd.isnull(X), axis=1)
print('Number failed: {}/{}'.format(np.sum(failed), len(failed)))

Number failed: 23/1056
